In [ ]:
%load_ext autoreload
%autoreload 2

from outlines import generate, models
from curverag import utils
from curverag.transformations import chunk_text
from curverag.graph import create_graph, KnowledgeGraph, Node, Edge, generate_prompt
import llama_cpp

In [ ]:
max_tokens = 5000
n_ctx=5000

docs = [
    "The patient was diagnosed with type 2 diabetes mellitus and prescribed metformin 500mg twice daily.",
    "MRI scan revealed a small lesion in the left temporal lobe suggestive of low-grade glioma.",
    "Administer 5mg of lorazepam intravenously for acute seizure management.",
    "Blood tests showed elevated ALT and AST levels, indicating possible liver inflammation.",
    "The subject reported chronic lower back pain, managed with physical therapy and NSAIDs.",
    "CT angiography confirmed the presence of a pulmonary embolism in the right lower lobe.",
    "The patient underwent coronary artery bypass graft surgery without complications.",
    "Routine vaccination included MMR, tetanus, and influenza immunizations.",
    "Histopathology indicated ductal carcinoma in situ (DCIS) in the breast biopsy sample.",
    "The child presented with a persistent cough and fever, diagnosed as streptococcal pharyngitis."
]


In [ ]:
model = utils.load_model(
    llm_model_path="models/Llama-3-13B-Instruct.i1-IQ3_S.gguf",
    tokenizer=llama_cpp.llama_tokenizer.LlamaHFTokenizer.from_pretrained("unsloth/Llama-3.2-1B-Instruct"),
    n_ctx=n_ctx,
    max_tokens=max_tokens
)

In [ ]:
graph = create_graph(model, docs, chunk_size=3000, max_tokens=max_tokens)